**Run McDonald-Krietman test using Biopython library and output MK p values and statistics (Dn, Ds, Pn, Ps).**

Codon alignment input generated in codon_alignment.ipynb. Analysis includes all Scer strains (Peter, et al 2018) with European S. paradoxus (Bergström et al., 2014) and North American S. paradoxus subpop B (Durand et al., 2019).

In [1]:
%load_ext autoreload
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.codonalign.codonseq import CodonSeq
from Bio.codonalign.codonalignment import CodonAlignment
from Bio.codonalign.codonalignment import mktest as MK

import os
import pandas as pd

/Users/clairedubin/opt/anaconda3/envs/py3/lib/python3.8/site-packages/Bio/codonalign/__init__.py:23: BiopythonExperimentalWarning: Bio.codonalign is an experimental module which may undergo significant changes prior to its future official release.
  warnings.warn(


In [2]:
#Some sequences have characters like "Y" and "R" to represent purines and pyrimidines
#Biopython doesn't recognize these, so change them to "N"

import re

def unambig_DNA_convert(seq):
    
    """Replace any ambiguous nucleotides with N"""
    
    return re.sub('[^ACGT]', 'N', seq)

Note: I modified "_G _test" in codonalignment.py to return NaN when there's a zero division error and stopped math domain errors by adding the "if obs == 0: continue" statement to the code below (can't take log of 0, but if obs=0 the value added to G would be 0 regardless):

    def _G_test(site_counts):
    """G test for 2x2 contingency table (PRIVATE).

    Arguments:
     - site_counts - [syn_fix, nonsyn_fix, syn_poly, nonsyn_poly]

    >>> print("%0.6f" % _G_test([17, 7, 42, 2]))
    0.004924
    """
    # TODO:
    #   Apply continuity correction for Chi-square test.
    from math import log

    # from scipy.stats import chi2
    try:
        G = 0
        tot = sum(site_counts)
        tot_syn = site_counts[0] + site_counts[2]
        tot_non = site_counts[1] + site_counts[3]
        tot_fix = sum(site_counts[:2])
        tot_poly = sum(site_counts[2:])
        exp = [
            tot_fix * tot_syn / tot,
            tot_fix * tot_non / tot,
            tot_poly * tot_syn / tot,
            tot_poly * tot_non / tot,
        ]
        for obs, ex in zip(site_counts, exp):
            if obs == 0:
                continue
            G += obs * log(obs / ex)
        G *= 2
        # return 1-chi2.cdf(G, 1) # only 1 dof for 2x2 table
        return chisqprob(G, 1)
    except:
        return "NaN"
        
I also added this statement in "mktest" at line 276 to drop codons with "N"s:
        
        all_codon = set([c for c in all_codons "N" not in c])

In [3]:
gap_cutoff = 0.1
percent_strains_present = 0.75

min_spar_seq_count = 20*0.75
min_scer_seq_count = 1011*0.75

In [4]:
rv_dict = {}

codon_dir = '/Users/clairedubin/sacc/carly_genes/alignments/1011Scer_allSpar_codon_alignments/codons_aligned/'

too_few_strains = []
errors = []

for f in os.listdir(codon_dir):
    
    spar_seqs, scer_seqs = [], []
    
    for record in SeqIO.parse(codon_dir+f, 'fasta'):
        
        seq = unambig_DNA_convert(str(record.seq))
        if len(seq(seq.count('-') + seq.count('N')) / len(seq) >= gap_cutoff:
            continue
        
        if '_Sp_' in record.id:
            spar_seqs += [SeqRecord(CodonSeq(seq), id=record.id)]
            
        else:
            scer_seqs += [SeqRecord(CodonSeq(seq), id=record.id)]
    
    if len(spar_seqs) < min_spar_seq_count or len(scer_seqs) < min_scer_seq_count:
        too_few_strains += [f]
        continue
    
    spar_codon_alignment = CodonAlignment(spar_seqs)
    scer_codon_alignment = CodonAlignment(scer_seqs)
    
    try:
        rv = MK([spar_codon_alignment, scer_codon_alignment])
        print(f, rv)
        rv_dict[f] = rv

    except:
        errors += [f]

YLL027W.pal2nal.fasta ([32, 7, 106, 86], 0.0011174462132920234)
YNL327W.pal2nal.fasta ([0, 0, 438, 457], 1.0)
YHR170W.pal2nal.fasta ([63, 1, 140, 44], 2.4960064585339753e-06)
YDL074C.pal2nal.fasta ([63, 17, 225, 171], 0.00015062229819909535)
YBR236C.pal2nal.fasta ([47, 12, 126, 69], 0.025226373906927908)
YBR223C.pal2nal.fasta ([36, 23, 127, 145], 0.04537728140580413)
YFR037C.pal2nal.fasta ([35, 2, 114, 37], 0.004075067691683954)
YER073W.pal2nal.fasta ([13, 0, 532, 386], 0.00017777530793159214)
YMR268C.pal2nal.fasta ([53, 28, 142, 156], 0.004230506098628952)
YOR320C.pal2nal.fasta ([61, 23, 178, 135], 0.007649029373065191)
YOR335C.pal2nal.fasta ([57, 1, 159, 42], 4.3847504113002267e-05)
YJL133W.pal2nal.fasta ([29, 0, 75, 32], 2.3943811568053914e-05)
YLR371W.pal2nal.fasta ([35, 3, 1154, 977], 3.0196190573469755e-07)
YJL126W.pal2nal.fasta ([33, 16, 109, 100], 0.05170018600156793)
YPL112C.pal2nal.fasta ([50, 8, 130, 106], 3.7892662896326285e-06)
YPR181C.pal2nal.fasta ([88, 2, 246, 43], 0.00

YKL085W.pal2nal.fasta ([39, 0, 124, 34], 4.4490553665058546e-05)
YJL209W.pal2nal.fasta ([39, 18, 298, 161], 0.5984458941242328)
YBR149W.pal2nal.fasta ([33, 2, 109, 69], 2.065037744478801e-05)
YKR016W.pal2nal.fasta ([54, 16, 171, 119], 0.0037137999997715163)
YKR003W.pal2nal.fasta ([43, 13, 104, 55], 0.10856077332367013)
YOL089C.pal2nal.fasta ([138, 34, 376, 375], 9.587882291051305e-14)
YOL141W.pal2nal.fasta ([99, 34, 287, 237], 2.5654915714262703e-05)
YDL156W.pal2nal.fasta ([53, 10, 216, 160], 2.1440919697012515e-05)
YCR045C.pal2nal.fasta ([71, 3, 173, 71], 4.078400149046234e-07)
YDL143W.pal2nal.fasta ([63, 1, 156, 31], 0.0002679008700057078)
YNL058C.pal2nal.fasta ([38, 11, 113, 94], 0.002452082922934885)
YGL107C.pal2nal.fasta ([90, 24, 192, 145], 1.5652279556995996e-05)
YGL112C.pal2nal.fasta ([70, 5, 134, 42], 0.0005595380023687171)
YGR244C.pal2nal.fasta ([55, 2, 137, 46], 4.8588325379117357e-05)
YMR282C.pal2nal.fasta ([61, 25, 143, 141], 0.0006425519930868482)
YER144C.pal2nal.fasta ([

YML123C.pal2nal.fasta ([12, 4, 154, 84], 0.3904175953202691)
YPR004C.pal2nal.fasta ([40, 10, 90, 93], 5.736045862500978e-05)
YPR011C.pal2nal.fasta ([46, 9, 97, 58], 0.0026928310845993637)
YNL172W.pal2nal.fasta ([217, 72, 648, 447], 3.7065252806059987e-07)
YLR225C.pal2nal.fasta ([56, 8, 135, 70], 0.00040857860920177197)
YJL072C.pal2nal.fasta ([22, 4, 65, 27], 0.1364103377841801)
YIL036W.pal2nal.fasta ([64, 29, 216, 254], 4.7090717071935696e-05)
YOR274W.pal2nal.fasta ([54, 18, 132, 88], 0.019060290168357263)
YHR031C.pal2nal.fasta ([74, 15, 271, 171], 3.5107025848627676e-05)
YHR024C.pal2nal.fasta ([58, 7, 193, 60], 0.01542768620872158)
YOL137W.pal2nal.fasta ([64, 11, 184, 121], 1.6422628860188003e-05)
YEL039C.pal2nal.fasta ([23, 0, 27, 6], 0.00890002043957835)
YKR060W.pal2nal.fasta ([34, 11, 73, 75], 0.0015091934115933677)
YGL164C.pal2nal.fasta ([51, 18, 175, 94], 0.1566628424321386)
YPR085C.pal2nal.fasta ([40, 36, 189, 193], 0.6153137876059602)
YPR158W.pal2nal.fasta ([40, 22, 78, 102], 0

YBR015C.pal2nal.fasta ([74, 23, 204, 129], 0.005281729229731958)
YGL186C.pal2nal.fasta ([71, 13, 206, 74], 0.03252240633992783)
YIR017C.pal2nal.fasta ([1, 0, 59, 31], 0.3597515262941424)
YIR002C.pal2nal.fasta ([84, 37, 317, 266], 0.0020256815465538834)
YBR248C.pal2nal.fasta ([68, 15, 179, 102], 0.0011734181839523755)
YJR046W.pal2nal.fasta ([62, 25, 275, 194], 0.02429925386995622)
YDR349C.pal2nal.fasta ([1, 0, 446, 593], 0.19358627020197927)
YKL189W.pal2nal.fasta ([36, 3, 127, 68], 0.00019474260777097797)
YJL090C.pal2nal.fasta ([82, 40, 259, 245], 0.0014573767116681038)
YMR196W.pal2nal.fasta ([130, 11, 335, 142], 6.752190724148887e-09)
YDR101C.pal2nal.fasta ([51, 5, 139, 101], 3.7728227354763786e-07)
YIL109C.pal2nal.fasta ([38, 1, 858, 544], 6.115456274734516e-08)
YOR283W.pal2nal.fasta ([16, 6, 42, 22], 0.5354388033559009)
YOR296W.pal2nal.fasta ([158, 41, 376, 286], 2.6216593966079458e-09)
YGR098C.pal2nal.fasta ([182, 73, 599, 477], 3.1978398187533764e-06)
YGR145W.pal2nal.fasta ([76, 14

YJR144W.pal2nal.fasta ([4, 4, 123, 69], 0.4266886724378096)
YBR282W.pal2nal.fasta ([23, 3, 64, 25], 0.0663992364578663)
YNL006W.pal2nal.fasta ([38, 0, 68, 5], 0.03783140718037479)
YGL084C.pal2nal.fasta ([72, 13, 200, 91], 0.0024621214324094874)
YDR402C.pal2nal.fasta ([30, 9, 156, 116], 0.01611075277463664)
YGL091C.pal2nal.fasta ([41, 2, 82, 29], 0.0009167691974485876)
YGL159W.pal2nal.fasta ([44, 23, 120, 146], 0.0024673204578671702)
YIL085C.pal2nal.fasta ([57, 20, 156, 110], 0.012451021470198217)
YLR283W.pal2nal.fasta ([42, 15, 89, 53], 0.13345777574545642)
YJL109C.pal2nal.fasta ([216, 48, 573, 339], 1.8513450287737624e-09)
YCL043C.pal2nal.fasta ([28, 5, 88, 46], 0.0241427466204774)
YOR147W.pal2nal.fasta ([63, 16, 184, 113], 0.00219872747451216)
YLR103C.pal2nal.fasta ([67, 12, 249, 128], 0.0005272559187066231)
YPR036W.pal2nal.fasta ([38, 3, 145, 37], 0.03284364670410472)
YML111W.pal2nal.fasta ([27, 1, 945, 827], 2.60922850844798e-07)
YNL088W.pal2nal.fasta ([189, 39, 341, 252], 1.355408

...

YER082C.pal2nal.fasta ([54, 17, 179, 96], 0.07292511711728676)
YKL113C.pal2nal.fasta ([41, 2, 130, 30], 0.012052969073171868)
YLR015W.pal2nal.fasta ([74, 19, 152, 109], 0.0001492757856067704)
YPL263C.pal2nal.fasta ([78, 7, 208, 114], 9.556540345713805e-08)
YOR211C.pal2nal.fasta ([130, 16, 302, 107], 6.1027324477431974e-05)
YPL023C.pal2nal.fasta ([85, 11, 197, 79], 0.0003468032547618075)
YMR104C.pal2nal.fasta ([27, 0, 661, 467], 9.799519292712847e-08)
YLR240W.pal2nal.fasta ([107, 6, 243, 126], 1.756461449464103e-11)
YMR111C.pal2nal.fasta ([25, 13, 119, 97], 0.2153826974975613)
YOR059C.pal2nal.fasta ([52, 8, 163, 93], 0.00025504604031284945)
YOR184W.pal2nal.fasta ([42, 6, 120, 58], 0.003574768432867835)
YOR191W.pal2nal.fasta ([202, 105, 583, 575], 1.1241362274984935e-06)
YER157W.pal2nal.fasta ([89, 21, 302, 261], 3.354951691229245e-08)
YBL051C.pal2nal.fasta ([64, 21, 230, 181], 0.0007022376778174678)
YMR284W.pal2nal.fasta ([80, 26, 194, 187], 3.904336304358043e-06)
YMR291W.pal2nal.fasta 

YNL299W.pal2nal.fasta ([79, 33, 194, 215], 1.09269344930481e-05)
YOL015W.pal2nal.fasta ([56, 73, 255, 308], 0.6980314309958109)
YDL017W.pal2nal.fasta ([53, 1, 160, 60], 1.8177465221842957e-06)
YBR255W.pal2nal.fasta ([63, 22, 268, 203], 0.0023025073717574136)
YDR289C.pal2nal.fasta ([45, 17, 112, 109], 0.001790686786722343)
YDR354W.pal2nal.fasta ([53, 21, 123, 71], 0.20063631442924965)
YER010C.pal2nal.fasta ([18, 4, 82, 76], 0.005716926323964891)
YKL194C.pal2nal.fasta ([60, 13, 162, 97], 0.0010392268637917728)
YLR092W.pal2nal.fasta ([33, 0, 902, 601], 8.203333088993953e-09)
YJL098W.pal2nal.fasta ([23, 4, 1035, 1180], 3.3788699680072624e-05)
YMR043W.pal2nal.fasta ([7, 1, 90, 38], 0.2607804614656736)
YNR007C.pal2nal.fasta ([31, 4, 82, 58], 0.0003661555690827596)
YOR356W.pal2nal.fasta ([73, 10, 221, 141], 5.610285259841815e-07)
YGR090W.pal2nal.fasta ([88, 13, 189, 99], 1.4442559795217114e-05)
YGR158C.pal2nal.fasta ([6, 7, 58, 58], 0.7924361680721359)
YNL081C.pal2nal.fasta ([25, 5, 51, 14], 

YDL135C.pal2nal.fasta ([21, 0, 56, 19], 0.0011156956483689996)
YER127W.pal2nal.fasta ([30, 10, 108, 105], 0.003724615309742504)
YBL034C.pal2nal.fasta ([8, 1, 27, 40], 0.004048456748581647)
YBL021C.pal2nal.fasta ([15, 1, 40, 8], 0.2644841972937652)
YGR232W.pal2nal.fasta ([15, 1, 94, 49], 0.00975932007013647)
YMR174C.pal2nal.fasta ([8, 4, 27, 39], 0.09840311923196104)
YPL053C.pal2nal.fasta ([40, 21, 126, 83], 0.4529912243850782)
YPL046C.pal2nal.fasta ([11, 3, 28, 38], 0.011865323409353017)
YIL023C.pal2nal.fasta ([45, 22, 123, 112], 0.02951119072608209)
YOR261C.pal2nal.fasta ([42, 8, 73, 33], 0.03883029004260587)
YPL206C.pal2nal.fasta ([26, 6, 117, 76], 0.018946448706832274)
YKL176C.pal2nal.fasta ([114, 21, 284, 185], 5.8893517779900894e-08)
YOR021C.pal2nal.fasta ([29, 4, 54, 36], 0.0019024348726590463)
YOR034C.pal2nal.fasta ([79, 37, 271, 231], 0.005121216881870747)
YDR036C.pal2nal.fasta ([0, 0, 160, 89], 1.0)
YPL193W.pal2nal.fasta ([33, 34, 138, 201], 0.19755493801275062)
YLR238W.pal2na

YER099C.pal2nal.fasta ([47, 3, 92, 24], 0.011113941817338836)
YGR019W.pal2nal.fasta ([8, 1, 177, 66], 0.24416487397168027)
YIL055C.pal2nal.fasta ([67, 30, 259, 231], 0.002905704981060476)
YDR180W.pal2nal.fasta ([173, 72, 523, 431], 5.695088142489102e-06)
YDR195W.pal2nal.fasta ([54, 15, 157, 150], 2.3072204519111087e-05)
YLR253W.pal2nal.fasta ([74, 8, 181, 90], 7.4564188993069854e-06)
YMR102C.pal2nal.fasta ([104, 17, 278, 149], 3.3280777371119638e-06)
YMR117C.pal2nal.fasta ([33, 10, 69, 45], 0.05214392799908731)
YBR094W.pal2nal.fasta ([71, 11, 267, 160], 6.588102471512534e-06)
YNL210W.pal2nal.fasta ([32, 32, 116, 116], 1.0)
YHR052W.pal2nal.fasta ([40, 14, 112, 83], 0.023474051557681404)
YNL185C.pal2nal.fasta ([15, 2, 53, 26], 0.062024898355743034)
YDR449C.pal2nal.fasta ([54, 12, 130, 93], 0.00027934378672606086)
YOR252W.pal2nal.fasta ([22, 0, 37, 27], 8.292532348370462e-06)
YIL010W.pal2nal.fasta ([6, 10, 92, 71], 0.14664751095488063)
YIL005W.pal2nal.fasta ([68, 49, 229, 216], 0.19815025

YIR024C.pal2nal.fasta ([17, 15, 81, 86], 0.6318379246092665)
YGL068W.pal2nal.fasta ([15, 3, 65, 40], 0.06368562284481243)
YBR026C.pal2nal.fasta ([37, 7, 142, 87], 0.002901713052843662)
YHR128W.pal2nal.fasta ([27, 1, 53, 17], 0.007135331979523806)
YGR163W.pal2nal.fasta ([50, 1, 107, 25], 0.0006541256570709073)
YPL097W.pal2nal.fasta ([60, 23, 161, 125], 0.00772975367647959)
YMR078C.pal2nal.fasta ([73, 26, 295, 260], 9.994586388283321e-05)
YOR125C.pal2nal.fasta ([12, 4, 39, 20], 0.4914386105309366)
YHL036W.pal2nal.fasta ([80, 14, 167, 93], 7.676084305198581e-05)
YOR130C.pal2nal.fasta ([40, 3, 85, 52], 2.2996541841270705e-05)
YLR174W.pal2nal.fasta ([54, 4, 114, 39], 0.0011585423760711408)
YBR175W.pal2nal.fasta ([41, 5, 104, 55], 0.0008355019944639829)
YBR160W.pal2nal.fasta ([40, 0, 85, 21], 0.00012364105892123184)
YFL037W.pal2nal.fasta ([43, 0, 82, 3], 0.11438582112445293)
YML020W.pal2nal.fasta ([5, 1, 527, 522], 0.0909156416485483)
YDR460W.pal2nal.fasta ([44, 2, 82, 30], 0.000384466781747

In [5]:
errors

[]

In [6]:
too_few_strains

['YGL075C.pal2nal.fasta',
 'YDL188C.pal2nal.fasta',
 'YNL086W.pal2nal.fasta',
 'YNL190W.pal2nal.fasta',
 'YLR105C.pal2nal.fasta',
 'YHL002W.pal2nal.fasta',
 'YJR054W.pal2nal.fasta',
 'YLR429W.pal2nal.fasta',
 'YGR129W.pal2nal.fasta',
 'YHR162W.pal2nal.fasta',
 'YDR341C.pal2nal.fasta',
 'YBR240C.pal2nal.fasta',
 'YJR053W.pal2nal.fasta',
 'YOR116C.pal2nal.fasta',
 'YJR100C.pal2nal.fasta',
 'YBR167C.pal2nal.fasta',
 'YKL133C.pal2nal.fasta',
 'YGR294W.pal2nal.fasta',
 'YGR281W.pal2nal.fasta',
 'YHL004W.pal2nal.fasta',
 'YJR052W.pal2nal.fasta',
 'YKL132C.pal2nal.fasta',
 'YBR166C.pal2nal.fasta',
 'YHR163W.pal2nal.fasta',
 'YKL069W.pal2nal.fasta',
 'YLR274W.pal2nal.fasta',
 'YGL047W.pal2nal.fasta',
 'YNL001W.pal2nal.fasta',
 'YPL271W.pal2nal.fasta',
 'YNL035C.pal2nal.fasta',
 'YDR496C.pal2nal.fasta',
 'YNL134C.pal2nal.fasta',
 'YPL273W.pal2nal.fasta',
 'YLR264W.pal2nal.fasta',
 'YOR265W.pal2nal.fasta',
 'YDL131W.pal2nal.fasta',
 'YPL240C.pal2nal.fasta',
 'YDR357C.pal2nal.fasta',
 'YML069W.pa

In [11]:
df_dict = {}

for f in rv_dict:
    
    df_dict[f.strip('.pal2nal.fasta')] = rv_dict[f][0] +[rv_dict[f][1]]
    
df = pd.DataFrame.from_dict(df_dict, orient='index')
df = df.reset_index()
df.columns = ['gene','Ds','Dn', 'Ps', 'Pn', 'p']
df.head()

,gene,Ds,Dn,Ps,Pn,p
0,YLL027W,32,7,106,86,0.001117
1,YNL327W,0,0,438,457,1.000000
2,YHR170W,63,1,140,44,0.000002
3,YDL074C,63,17,225,171,0.000151
4,YBR236C,47,12,126,69,0.025226


In [12]:
df.to_csv('MK_stats_AllScer_AllSpar.csv', index=False)

In [9]:
df.shape

(3855, 5)